# Geoscientific Plotting



In [ ]:
# Some default settings
import matplotlib.pyplot as plt

plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.constrained_layout.use"] = True


## Plotting with Cartopy

Let's plot a nice map of the Earth in a coordinate system of choice!

Docs: https://scitools.org.uk/cartopy/docs/v0.21/index.html

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt


def create_maps(add_stock_img=True):
    fig = plt.figure()

    CRS = [
        ccrs.PlateCarree(),
        ccrs.Mollweide(),
        ccrs.NearsidePerspective(central_longitude=-30, central_latitude=30),
        ccrs.InterruptedGoodeHomolosine(),
    ]
    for idx, crs in enumerate(CRS):
        ax = fig.add_subplot(2, 2, idx + 1, projection=crs)
        ax.set_title(type(crs).__name__)
        if add_stock_img:
            ax.stock_img()

    return fig


fig = create_maps()


In [ ]:
new_york = [40.71, -74.01]
zurich = [47.37, 8.54]

fig = create_maps()
for ax in fig.axes:
    ax.plot(
        [new_york[1], zurich[1]],
        [new_york[0], zurich[0]],
        "-k",
        transform=ccrs.PlateCarree(),
    )
    ax.plot(
        [new_york[1], zurich[1]],
        [new_york[0], zurich[0]],
        "-r",
        transform=ccrs.Geodetic(),
    )


Try out more of the cartopy features

In [ ]:
import cartopy.feature as cfeature

fig = create_maps(add_stock_img=False)
axes = fig.axes

# Plot coastlines and the lat/lon grid with labels
axes[0].coastlines()
axes[0].gridlines(draw_labels=True)

# Chosse a map extent in lat/lon coordinates, plot features
axes[1].set_extent([80, 170, -45, 30], crs=ccrs.PlateCarree())
axes[1].add_feature(cfeature.LAND)
axes[1].add_feature(cfeature.COASTLINE)
axes[1].add_feature(cfeature.RIVERS)
axes[1].add_feature(cfeature.LAKES)
axes[1].gridlines(draw_labels=["y"], y_inline=True)

# Change scale and appearance of a feature
axes[2].add_feature(
    cfeature.COASTLINE.with_scale("50m"), lw=0.5, edgecolor="black", facecolor="none"
)

# Download a particular feature from NaturalEarth
borders = cfeature.NaturalEarthFeature("cultural", "admin_0_boundary_lines_land", "50m")
axes[2].add_feature(borders, lw=0.5, edgecolor="grey", facecolor="none")
axes[2].add_feature(cfeature.OCEAN)

# Projection emphasizing oceans
fig.axes[3].remove()
fig.axes[3] = fig.add_subplot(
    2, 2, 4, projection=ccrs.InterruptedGoodeHomolosine(emphasis="ocean")
)
fig.axes[3].add_feature(cfeature.OCEAN)


## Plotting data with xarray and cartopy

Open a geoscientific dataset with xarray.
The dataset contains daily mean temperatures that were observed in Europe between 2011-01-01 and 2011-01-31.
This data can be downloaded from the Copernicus Climate Data Store at https://cds.climate.copernicus.eu/cdsapp#!/dataset/insitu-gridded-observations-europe

Docs: https://docs.xarray.dev/en/stable/

In [ ]:
import xarray as xr

ds = xr.open_dataset("tas_2011_01.nc")
ds


Plotting this data naively with the xarray plotting facilities already looks quite reasonable.

In [ ]:
da = ds["tas"].sel(time="2011-01-01")
da.plot()


The quality of the plot changes quite drastically if the spatial dimensions of the dataset are changed.
This just follows the matplotlib default settings: Each axis is extended until the figure pane is filled.

In [ ]:
da.sel(longitude=slice(0, 20)).plot()


Cartopy fixes the aspect to the default for the chosen projection and removes the ticks and tick labels by default.

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
da.sel(longitude=slice(0, 20)).plot(ax=ax)
ax.add_feature(cfeature.COASTLINE)


Let's look at the data through different projections!

In [ ]:
fig = plt.figure()

CRS = [
    ccrs.PlateCarree(),
    ccrs.Miller(),
    ccrs.Mercator(),
    ccrs.EuroPP(),
]
for idx, crs in enumerate(CRS):
    ax = fig.add_subplot(2, 2, idx + 1, projection=crs)

    da.plot(ax=ax, add_colorbar=False, transform=ccrs.PlateCarree())
    ax.set_title(type(crs).__name__)
    ax.add_feature(cfeature.COASTLINE, lw=0.5)
    ax.gridlines(draw_labels=["bottom", "left"])


## Plotting data with GeoPandas

Although GeoPandas provides some plotting facilities, it takes a much more "naive" approach.
While GeoPandas can transform geometries between different coordinate systems and projections, it always plots the data as is: There is no adjustment of matplotlib defaults and there are also no transformations (by default).
The only exception: The figure aspect is set to `equal` if no transformation is applied to the data.

Docs: https://geopandas.org/en/stable/

Let's download some data for Luxembourg from GeoFabrik and plot it!

In [ ]:
from pathlib import Path

import osm_flex.download as dl
import osm_flex.extract as ex

# Download GeoFabrik data for Switzerland
dl.get_country_geofabrik("LUX", save_path=Path("./"))

# Extract main roads and railways
data_path = Path("luxembourg-latest.osm.pbf")
lux_railway = ex.extract_cis(data_path, "rail").set_geometry("geometry")
lux_railway


In GeoPandas, coordinate systems and projections are identified by their EPSG code.
The most basic coordinate system is the WGS 84, a lat/lon system used by GPS.
It's EPSG code is 4326.
Most geospatial data is supplied in this system, and it's the same for the GeoFabrik data:

In [ ]:
lux_railway.geometry.crs


Let's plot the data. Note the nice aspect figure!

In [ ]:
fig, ax = plt.subplots()
lux_railway.loc[lux_railway["railway"] == "rail"].geometry.plot(
    ax=ax, color="C0", label="Rails", zorder=1
)
lux_railway.loc[lux_railway["railway"] == "station"].geometry.plot(
    ax=ax, color="C1", label="Stations", markersize=10, zorder=2
)
ax.legend()


It is important to note that WGS 84 corresponds to the cartopy `PlateCarree` projection.

In [ ]:
proj = ccrs.Mercator()
fig, ax = plt.subplots(subplot_kw=dict(projection=proj))

lux_railway.loc[lux_railway["railway"] == "rail"].geometry.plot(
    ax=ax,
    color="C0",
    label="Rails",
    zorder=1,
    transform=ccrs.PlateCarree(),
    aspect=None,
)
lux_railway.loc[lux_railway["railway"] == "station"].geometry.plot(
    ax=ax,
    color="C1",
    label="Stations",
    markersize=10,
    zorder=2,
    transform=ccrs.PlateCarree(),
    aspect=None,
)
ax.legend()
ax.add_feature(cfeature.BORDERS.with_scale("10m"))
ax.gridlines(draw_labels=True, zorder=0)


If you are unsure about the projection, you can simply transform the GeoPandas data by using the `proj4_params` attribute of the cartopy projection.

Let's combine the railway data with the xarray data plot!

In [ ]:
proj = ccrs.Miller()
fig, ax = plt.subplots(subplot_kw=dict(projection=proj))

lux_railway.loc[lux_railway["railway"] == "rail"].geometry.to_crs(
    proj.proj4_params
).plot(
    ax=ax,
    color="k",
    label="Rails",
    zorder=1,
    aspect=None,
)
lux_railway.loc[lux_railway["railway"] == "station"].geometry.to_crs(
    proj.proj4_params
).plot(
    ax=ax,
    color="white",
    label="Stations",
    markersize=10,
    edgecolor="black",
    zorder=2,
    aspect=None,
)

# Retrieve current extent in PlateCarree projection (WGS 84)
extent = ax.get_extent(crs=ccrs.PlateCarree())

# Now plot temperature for that extent
da.sel(
    longitude=slice(extent[0], extent[1]), latitude=slice(extent[2], extent[3])
).plot(ax=ax, transform=ccrs.PlateCarree(), zorder=0)

ax.legend()
ax.add_feature(cfeature.BORDERS.with_scale("10m"))
ax.gridlines(draw_labels=["left", "bottom"], zorder=0)


## Plotting Maps with contextily

Contextily offers basemaps that can serve as better geographical reference than coordinates and country borders (alone).
It works with the same projection definition as GeoPandas.

Docs: https://contextily.readthedocs.io/

In [ ]:
import contextily as ctx

fig, ax = plt.subplots()


lux_railway.loc[lux_railway["railway"] == "rail"].geometry.plot(
    ax=ax,
    color="k",
    label="Rails",
    zorder=1,
)
lux_railway.loc[lux_railway["railway"] == "station"].geometry.plot(
    ax=ax,
    color="white",
    label="Stations",
    markersize=10,
    edgecolor="black",
    zorder=2,
)

ctx.add_basemap(ax=ax, source=ctx.providers.OpenStreetMap.Mapnik, crs=lux_railway.crs)
ax.legend()


This looks distorted, because maps typically use the Mercator projection, with is conformal and therefore preserves local shapes.
The EPSG code of the Mercator projection is 3857.
Transforming our data into this projection should make it look much better

In [ ]:
fig, ax = plt.subplots()

lux_railway_plot = lux_railway.to_crs("EPSG:3857")  # Transform to Mercator
lux_railway_plot.loc[lux_railway["railway"] == "rail"].geometry.plot(
    ax=ax,
    color="k",
    label="Rails",
    zorder=1,
)
lux_railway_plot.loc[lux_railway["railway"] == "station"].geometry.plot(
    ax=ax,
    color="white",
    label="Stations",
    markersize=10,
    edgecolor="black",
    zorder=2,
)
ax.legend()

ctx.add_basemap(
    ax=ax, source=ctx.providers.OpenStreetMap.Mapnik, crs=lux_railway_plot.crs
)


The ticks are quite annoying.
There are two ways to fix them: Remove the ticks from the axis, or use cartopy to set nice defaults instead.

In [ ]:
fig, ax = plt.subplots()

lux_railway_plot = lux_railway.to_crs("EPSG:3857")  # Transform to Mercator
lux_railway_plot.loc[lux_railway_plot["railway"] == "rail"].geometry.plot(
    ax=ax,
    color="k",
    label="Rails",
    zorder=1,
)
lux_railway_plot.loc[lux_railway_plot["railway"] == "station"].geometry.plot(
    ax=ax,
    color="white",
    label="Stations",
    markersize=10,
    edgecolor="black",
    zorder=2,
)

# CRS not strictly necessary because the map is in Mercator projection anyway
ctx.add_basemap(
    ax=ax, source=ctx.providers.OpenStreetMap.Mapnik, crs=lux_railway_plot.crs
)
ax.legend()

# Remove ticks
ax.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)


Use a map with separate labels and less colors, this gives more opportunities for introducing color yourself

In [ ]:
fig, ax = plt.subplots()

lux_railway_plot = lux_railway.to_crs("EPSG:3857")  # Transform to Mercator
lux_railway_plot.loc[lux_railway_plot["railway"] == "rail"].geometry.plot(
    ax=ax,
    color="C0",
    label="Rails",
    zorder=1,
)
lux_railway_plot.loc[lux_railway_plot["railway"] == "station"].geometry.plot(
    ax=ax,
    color="C1",
    label="Stations",
    markersize=10,
    zorder=2,
)

# CRS not strictly necessary because the map is in Mercator projection anyway
ctx.add_basemap(
    ax=ax, source=ctx.providers.CartoDB.PositronNoLabels, crs=lux_railway_plot.crs
)
ctx.add_basemap(
    ax=ax, source=ctx.providers.CartoDB.PositronOnlyLabels, crs=lux_railway_plot.crs
)
ax.legend()

# Remove ticks
ax.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)


Now let's combine cartopy, GeoPandas and contextily!

In [ ]:
proj = ccrs.Mercator()
fig, ax = plt.subplots(subplot_kw=dict(projection=proj))

ax.set_xmargin(0.5)
ax.set_ymargin(0.5)

lux_railway_plot = lux_railway.to_crs(proj.proj4_params)  # Transform to Mercator
lux_railway_plot.loc[lux_railway_plot["railway"] == "rail"].geometry.plot(
    ax=ax,
    color="black",
    label="Rails",
    zorder=2,
    aspect=None,
)
lux_railway_plot.loc[lux_railway_plot["railway"] == "station"].geometry.plot(
    ax=ax,
    color="white",
    edgecolor="black",
    label="Stations",
    markersize=10,
    zorder=3,
    aspect=None,
)
extent = ax.get_extent(crs=ccrs.PlateCarree())

# Now plot temperature for that extent
da.sel(
    longitude=slice(extent[0], extent[1]), latitude=slice(extent[2], extent[3])
).plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=10, alpha=0.4, zorder=1)

# CRS not strictly necessary because the map is in Mercator projection anyway
ctx.add_basemap(
    ax=ax, source=ctx.providers.CartoDB.PositronNoLabels, crs=proj.proj4_params
)
ctx.add_basemap(
    ax=ax, source=ctx.providers.CartoDB.PositronOnlyLabels, crs=proj.proj4_params
)

ax.legend()
